# **Basics of scikit-learn package** 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
from plotly.offline import init_notebook_mode
import warnings

init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
sns.set_theme(style='whitegrid')

## Supervised learning (контролируемое обучение / обучение с учителем)

### 1.1 *Linear Models (Линейные модели)*

Линейные модели являются основой многих методов машинного обучения. Они просты в интерпретации, быстры в обучении и хорошо работают на больших данных. Основная идея линейных моделей заключается в том, что целевая переменная $y$ выражается как линейная комбинация входных признаков $X$:

$$
y = w_0 + w_1 x_1 + w_2 x_2 + \dots + w_n x_n + \varepsilon
$$

где:
- $w_0$ — свободный коэффициент (intercept),
- $w_1, \dots, w_n$ — веса модели (коэффициенты),
- $x_1, \dots, x_n$ — входные признаки,
- $\varepsilon$ — ошибка модели (остаточный член).

### 1.1.1 Метод наименьших квадратов (Ordinary Least Squares, OLS)

Метод наименьших квадратов минимизирует сумму квадратов ошибок между предсказанными значениями $\hat{y}$ и реальными значениями $y$:

$$
J(w) = \sum_{i=1}^{m} (y_i - \hat{y}_i)^2 = \sum_{i=1}^{m} (y_i - (w_0 + \sum_{j=1}^{n} w_j x_{ij}))^2
$$

Оптимальные коэффициенты $w$ находятся по формуле:

$$
\mathbf{w} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$

В scikit-learn линейная регрессия реализуется с помощью `LinearRegression`:

In [2]:
from sklearn.linear_model import LinearRegression

X = [[1, 2], [2, 3], [3, 4], [4, 5]]  # Признаки
y = [3, 5, 7, 9]  # Целевая переменная

model = LinearRegression()
model.fit(X, y)

print("Коэффициенты:", model.coef_)
print("Свободный член:", model.intercept_)

Коэффициенты: [1. 1.]
Свободный член: 1.7763568394002505e-15


Оценки коэффициентов для метода наименьших квадратов основаны на независимости признаков. Когда признаки коррелируют и столбцы матрицы признаков $X$
имеют приблизительно линейную зависимость, матрица  становится близкой к сингулярной, и в результате оценка наименьших квадратов становится очень чувствительной к случайным ошибкам в наблюдаемой цели, производя большую дисперсию. Такая ситуация мультиколлинеарности может возникнуть, например, когда данные собираются без экспериментального дизайна.

Можно ограничить все коэффициенты неотрицательными значениями, что может быть полезно, когда они представляют некоторые физические или естественно неотрицательные величины (например, частотные показатели или цены товаров). `LinearRegression` принимает булевый параметр `positive` : если установлено значение `True` , то применяются наименьшие квадраты.

### 1.1.2 Линейная регрессия с L2-регуляризацией (Ridge Regression)

Ridge-регрессия добавляет L2-регуляризацию (штраф за большие коэффициенты) для предотвращения переобучения:

$$
J(w) = \sum_{i=1}^{m} (y_i - \hat{y}_i)^2 + \lambda \sum_{j=1}^{n} w_j^2
$$

где $\lambda$ — параметр регуляризации. Чем больше $\lambda$, тем сильнее штраф за большие коэффициенты.

Пример реализации Ridge-регрессии в scikit-learn:

In [3]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X, y)

print("Коэффициенты Ridge:", ridge.coef_)

Коэффициенты Ridge: [0.90909091 0.90909091]


Обратите внимание, что класс `Ridge` позволяет пользователю указать, что решатель будет выбран автоматически, установив `solver="auto"`. Если указан этот параметр, `Ridge` будет выбирать между решателями `"lbfgs"`, `"cholesky"` и `"sparse_cg"`. `Ridge` начнет проверять условия, показанные в следующей таблице сверху вниз. Если условие истинно, выбирается соответствующий решатель.

| Решатель | Состояние |
|---------|------------|
| `'lbfgs'`    | Параметр `positive=True` указана. |
| `"cholesky"` | Входной массив X не является разреженным. |
| `'sparse_cg'` | Ни одно из вышеперечисленных условий не выполнено. |
---

### 1.1.3 Линейная регрессия с L1-регуляризацией (Lasso Regression)

Lasso-регрессия использует L1-регуляризацию, накладывая штраф на сумму абсолютных значений коэффициентов:

$$
J(w) = \sum_{i=1}^{m} (y_i - \hat{y}_i)^2 + \lambda \sum_{j=1}^{n} |w_j|
$$

Lasso способствует разреженности модели, зануляя некоторые коэффициенты, что полезно для отбора признаков.

Параметр `alpha` контролирует степень разреженности оценочных коэффициентов.

Пример в scikit-learn:

In [4]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(X, y)

print("Коэффициенты Lasso:", lasso.coef_)

Коэффициенты Lasso: [1.92000000e+00 1.33226763e-16]


### 1.1.4 Эластичная сеть (Elastic Net)

Elastic Net сочетает L1- и L2-регуляризации:

$$
J(w) = \sum_{i=1}^{m} (y_i - \hat{y}_i)^2 + \lambda_1 \sum_{j=1}^{n} |w_j| + \lambda_2 \sum_{j=1}^{n} w_j^2
$$

Этот метод особенно полезен, если признаки коррелированы между собой.

Пример Elastic Net в scikit-learn:

In [5]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X, y)

print("Коэффициенты Elastic Net:", elastic_net.coef_)

Коэффициенты Elastic Net: [0.96195901 0.9596548 ]


---

### 1.1.5 Логистическая регрессия (Logistic Regression)

Логистическая регрессия используется для задач классификации и моделирует вероятность принадлежности объекта к определённому классу с помощью сигмоидной функции:

$$
P(y=1 | x) = \frac{1}{1 + e^{- (w_0 + w_1 x_1 + ... + w_n x_n)}}
$$

Функция потерь для логистической регрессии — логарифмическая:

$$
J(w) = - \sum_{i=1}^{m} \left[ y_i \log \hat{y}_i + (1 - y_i) \log (1 - \hat{y}_i) \right]
$$

In [6]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X, y)

print("Коэффициенты:", log_reg.coef_)

Коэффициенты: [[-0.562841   -0.56289585]
 [-0.16786972 -0.16808781]
 [ 0.16793833  0.16799842]
 [ 0.5627724   0.56298524]]


### 1.1.5 Байесовская регрессия (Bayesian Regression)

Байесовская регрессия учитывает априорное распределение параметров модели, позволяя получать интервальные оценки коэффициентов.

Пример использования в scikit-learn:

In [7]:
from sklearn.linear_model import BayesianRidge

bayes_ridge = BayesianRidge()
bayes_ridge.fit(X, y)

print("Коэффициенты Bayesian Ridge:", bayes_ridge.coef_)

Коэффициенты Bayesian Ridge: [0.99999997 0.99999997]
